In [0]:
!pip install mlflow


Requirement already satisfied: mlflow in /databricks/python3/lib/python3.8/site-packages (1.19.0)
Requirement already satisfied: Flask in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.0.1)
Requirement already satisfied: gunicorn; platform_system != "Windows" in /databricks/python3/lib/python3.8/site-packages (from mlflow) (20.1.0)
Requirement already satisfied: sqlparse>=0.3.1 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.4.1)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from mlflow) (21.0)
Requirement already satisfied: databricks-cli>=0.8.7 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.14.3)
Requirement already satisfied: requests>=2.17.3 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2.24.0)
Requirement already satisfied: sqlalchemy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.4.21)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.8/site-packages (from mlflow) (2020.5)
Requirement already satisfied: protobuf>=3.7.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.17.0)
Requirement already satisfied: docker>=4.0.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (5.0.0)
Requirement already satisfied: cloudpickle in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.6.0)
Requirement already satisfied: numpy in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.19.2)
Requirement already satisfied: pyyaml>=5.1 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (5.4.1)
Requirement already satisfied: alembic<=1.4.1 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.4.1)
Requirement already satisfied: pandas in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.1.5)
Requirement already satisfied: entrypoints in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.3)
Requirement already satisfied: gitpython>=2.1.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (3.1.18)
Requirement already satisfied: querystring-parser in /databricks/python3/lib/python3.8/site-packages (from mlflow) (1.2.4)
Requirement already satisfied: prometheus-flask-exporter in /databricks/python3/lib/python3.8/site-packages (from mlflow) (0.18.2)
Requirement already satisfied: click>=7.0 in /databricks/python3/lib/python3.8/site-packages (from mlflow) (8.0.1)
Requirement already satisfied: itsdangerous>=2.0 in /databricks/python3/lib/python3.8/site-packages (from Flask->mlflow) (2.0.1)
Requirement already satisfied: Werkzeug>=2.0 in /databricks/python3/lib/python3.8/site-packages (from Flask->mlflow) (2.0.1)
Requirement already satisfied: Jinja2>=3.0 in /databricks/python3/lib/python3.8/site-packages (from Flask->mlflow) (3.0.1)
Requirement already satisfied: setuptools>=3.0 in /databricks/python3/lib/python3.8/site-packages (from gunicorn; platform_system != "Windows"->mlflow) (50.3.1)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.8/site-packages (from packaging->mlflow) (2.4.7)
Requirement already satisfied: six>=1.10.0 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli>=0.8.7->mlflow) (1.15.0)
Requirement already satisfied: tabulate>=0.7.7 in /databricks/python3/lib/python3.8/site-packages (from databricks-cli>=0.8.7->mlflow) (0.8.9)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (2020.12.5)
Requirement already satisfied: chardet<4,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (3.0.4)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests>=2.17.3->mlflow) (2.10)
Requirement already satisfied: urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1 in /databricks/python3/lib/python3.8/site-package

In [0]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql import Row
from pyspark.sql.functions import when
from pyspark.ml.feature import StringIndexer, IndexToString
import mlflow
import mlflow.spark

In [0]:
df = spark.read.csv('dbfs:/FileStore/tables/lastfm-data/usersha1-artmbid-artname-plays.tsv', sep='\t')
df.show(5)

+--------------------+--------------------+--------------------+----+
 _c0| _c1| _c2| _c3|
+--------------------+--------------------+--------------------+----+
00000c289a1829a80...|3bd73256-3905-4f3...| betty blowtorch|2137|
00000c289a1829a80...|f2fb0ff0-5679-42e...| die Ärzte|1099|
00000c289a1829a80...|b3ae82c2-e60b-455...| melissa etheridge| 897|
00000c289a1829a80...|3d6bbeb7-f90e-4d1...| elvenking| 717|
00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks| 706|
+--------------------+--------------------+--------------------+----+
only showing top 5 rows

In [0]:
df = df.dropna()

In [0]:
newNames = ['userid', 'artistid', 'artistname', 'plays']
df = df.toDF(*newNames)
df.show(5)

+--------------------+--------------------+--------------------+-----+
 userid| artistid| artistname|plays|
+--------------------+--------------------+--------------------+-----+
00000c289a1829a80...|3bd73256-3905-4f3...| betty blowtorch| 2137|
00000c289a1829a80...|f2fb0ff0-5679-42e...| die Ärzte| 1099|
00000c289a1829a80...|b3ae82c2-e60b-455...| melissa etheridge| 897|
00000c289a1829a80...|3d6bbeb7-f90e-4d1...| elvenking| 717|
00000c289a1829a80...|bbd2ffd7-17f4-450...|juliette & the licks| 706|
+--------------------+--------------------+--------------------+-----+
only showing top 5 rows

In [0]:
# label encode users and artists
userIndexer = StringIndexer(inputCol="userid", outputCol="userIndex")
artistIndexer = StringIndexer(inputCol="artistid", outputCol="artistIndex")

userDecoder = IndexToString(inputCol="userIndex", outputCol="useridOrignial")
artistDecoder = IndexToString(inputCol="artistIndex", outputCol="artistidOriginal")

df = userIndexer.setHandleInvalid("keep").fit(df).transform(df) # sethandleInvalid("keep") because randomSplit was throwing error about string Indexer. Null values?
df = artistIndexer.setHandleInvalid("keep").fit(df).transform(df)

In [0]:
display(df.take(5))

userid,artistid,artistname,plays,userIndex,artistIndex
00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137,126839.0,17191.0
00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099,126839.0,277.0
00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897,126839.0,3120.0
00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717,126839.0,1981.0
00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706,126839.0,2121.0


In [0]:
df2 = df.withColumn("ratings", when(df.plays>50, 1).otherwise(0))
# df2.describe().show()

In [0]:
# data = df.select("userIndex", "artistIndex", "plays")
# data = df.selectExpr("INT(userIndex)","INT(artistIndex)","FLOAT(plays)","FLOAT(ratings)", "userid", "artistId")
data_demo = df2.selectExpr("INT(userIndex)","INT(artistIndex)","FLOAT(ratings)")

data_demo.dtypes

Out[8]: [('userIndex', 'int'), ('artistIndex', 'int'), ('ratings', 'float')]

In [0]:
(demo, rest) = data_demo.randomSplit([0.1, 0.9])

In [0]:
mlflow_set_experiment(experiment_name ="lastfm-als-experiment1",
                     artifact_location = "/Users/sengarnh@mail.uc.edu/mxm/")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-2558517743205188> in <module> 
 ----> 1 mlflow_set_experiment(experiment_name ="lastfm-als-experiment1",
 2 artifact_location = "/Users/sengarnh@mail.uc.edu/mxm/")

 NameError : name 'mlflow_set_experiment' is not defined

In [0]:
# source: docs 
# ratingsRDD = parts.map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))
# ratings = spark.createDataFrame(ratingsRDD)



alphas = [0.1, 0.5, 1]
ranks = [5, 10, 15]
for alpha in alphas:
  for rank in ranks:
    (training, test) = demo.randomSplit([0.8, 0.2])
    with mlflow.start_run(run_name='lastfm-als'):

      als = ALS(userCol="userIndex", itemCol="artistIndex", ratingCol="ratings",
                implicitPrefs=True)
      als.setMaxIter(5)
      als.setRegParam(0.1)
      als.setRank(rank)
      als.setAlpha(alpha)

      mlflow.log_param("model", "als")
      mlflow.log_param("maxIter", als.getMaxIter())
      mlflow.log_param("regParam", als.getRegParam())
      mlflow.log_param("rank", rank)
      mlflow.log_param("alpha", alpha)
      mlflow.log_param("implicitPrefs", als.getImplicitPrefs())

      model = als.fit(training)
      predictions = model.transform(test)
      predictions = predictions.filter(predictions.prediction != float('nan'))
      evaluator = RegressionEvaluator(metricName="rmse", labelCol="ratings",predictionCol="prediction")
      rmse = evaluator.evaluate(predictions)
      mlflow.log_metric("rmse", rmse)
      print("RANK= %s :Root-mean-square error = %s" %(str(rank), str(rmse)))

RANK= 5 :Root-mean-square error = 0.818411144457406
RANK= 10 :Root-mean-square error = 0.8191470517639242
RANK= 15 :Root-mean-square error = 0.8187125598742055
RANK= 5 :Root-mean-square error = 0.8181080202245553
RANK= 10 :Root-mean-square error = 0.8180307222996517
RANK= 15 :Root-mean-square error = 0.8177529099192312
RANK= 5 :Root-mean-square error = 0.8177355793716493
RANK= 10 :Root-mean-square error = 0.816826310656652
RANK= 15 :Root-mean-square error = 0.8156793180932668

In [0]:
run_id = mlflow.search_runs(filter_string='tags.mlflow.runName = "lastfm-als"').iloc[0].run_id

model_name = "lastfm-als-model"

model_version = mlflow.register_model(f"runs:/{run_id}/lastfm-als", model_name)

--------------------------------------------------------------------------- 
 InvalidConfigurationError Traceback (most recent call last)
 <command-440421022018919> in <module> 
 ----> 1 run_id = mlflow . search_runs ( filter_string = 'tags.mlflow.runName = "lastfm-als"' ) . iloc [ 0 ] . run_id
 2 
 3 model_name = "lastfm-als-model" 
 4 
 5 model_version = mlflow . register_model ( f"runs:/{run_id}/lastfm-als" , model_name ) 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/fluent.py in search_runs (experiment_ids, filter_string, run_view_type, max_results, order_by, output_format) 
 1061 )
 1062 
 -> 1063 runs = _paginate ( pagination_wrapper_func , NUM_RUNS_PER_PAGE_PANDAS , max_results ) 
 1064 
 1065 if output_format == "list" : 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/fluent.py in _paginate (paginated_fn, max_results_per_page, max_results) 
 1232 page_results = paginated_fn ( num_to_get , next_page_token ) 
 1233 else : 
 -> 1234 page_results = paginated_fn ( max_results_per_page , next_page_token ) 
 1235 all_results . extend ( page_results ) 
 1236 if hasattr ( page_results , "token" ) and page_results . token : 

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/fluent.py in pagination_wrapper_func (number_to_get, next_page_token) 
 1057 # full thing is a mess 
 1058 def pagination_wrapper_func ( number_to_get , next_page_token ) : 
 -> 1059 return MlflowClient().search_runs(
 1060 experiment_ids , filter_string , run_view_type , number_to_get , order_by , next_page_token , 
 1061 )

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/client.py in search_runs (self, experiment_ids, filter_string, run_view_type, max_results, order_by, page_token) 
 1588 tags : { 's.release' : '1.1.0-RC' } 
 1589 """
 -> 1590 return self._tracking_client.search_runs(
 1591 experiment_ids , filter_string , run_view_type , max_results , order_by , page_token
 1592 )

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_tracking_service/client.py in search_runs (self, experiment_ids, filter_string, run_view_type, max_results, order_by, page_token) 
 407 if isinstance ( experiment_ids , int ) or is_string_type ( experiment_ids ) : 
 408 experiment_ids = [ experiment_ids ] 
 --> 409 return self.store.search_runs(
 410 experiment_ids = experiment_ids , 
 411 filter_string = filter_string , 

 /databricks/python/lib/python3.8/site-packages/mlflow/store/tracking/abstract_store.py in search_runs (self, experiment_ids, filter_string, run_view_type, max_results, order_by, page_token) 
 240 meaningful in such cases . 
 241 """
 --> 242 runs, token = self._search_runs(
 243 experiment_ids , filter_string , run_view_type , max_results , order_by , page_token
 244 )

 /databricks/python/lib/python3.8/site-packages/mlflow/store/tracking/rest_store.py in _search_runs (self, experiment_ids, filter_string, run_view_type, max_results, order_by, page_token) 
 257 )
 258 req_body = message_to_json ( sr ) 
 --> 259 response_proto = self . _call_endpoint ( SearchRuns , req_body ) 
 260 runs = [ Run . from_proto ( proto_run ) for proto_run in response_proto . runs ] 
 261 # If next_page_token is not set, we will see it as "". We need to convert this to None. 

 /databricks/python/lib/python3.8/site-packages/mlflow/store/tracking/rest_store.py in _call_endpoint (self, api, json_body) 
 54 endpoint , method = _METHOD_TO_INFO [ api ] 
 55 response_proto = api . Response ( ) 
 ---> 56 return call_endpoint ( self . get_host_creds ( ) , endpoint , method , json_body , response_proto ) 
 57 
 58 def list_experiments(

 /databricks/python/lib/python3.8/site-packages/mlflow/tracking/_tracking_service/utils.py in <lambda> () 
 135 
 136 def _get_databricks_rest_store ( store_uri , ** _ ) : 
 --> 137 return DatabricksRestStore ( lambda : get_databricks_host_creds ( store_uri ) ) 
 138 
 139 

 /databricks/python/lib/python3.8/site-packages/mlflow/utils/databricks_utils.py in get_databricks_host_

In [0]:
# # Generate top 10 movie recommendations for each user
# userRecs = model.recommendForAllUsers(10)
# # Generate top 10 user recommendations for each movie
# movieRecs = model.recommendForAllItems(10)

In [0]:
rmse = evaluator.evaluate(predictions)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4142293834736426> in <module> 
 ----> 1 rmse = evaluator . evaluate ( predictions ) 

 NameError : name 'evaluator' is not defined

In [0]:
rmse

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4142293834736431> in <module> 
 ----> 1 rmse

 NameError : name 'rmse' is not defined

In [0]:
model.save(sc, "./model/als-mlflow")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-4142293834736427> in <module> 
 ----> 1 model . save ( sc , "./model/als-mlflow" ) 

 NameError : name 'model' is not defined